In [1]:
import numpy as np

import sys
sys.path.insert(
    1,
    '/home/trduong/Data/interpretable_machine_learning/Source Code/my_work/lib'
)

import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import utils


from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from geneticalgorithm import geneticalgorithm as ga


In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_path)

In [3]:
train_data['treatment'] = np.where(train_data['price'] == 1, 1, 0)
train_data['price'] = np.where(train_data['price'] == 1, 1, 0.85)

In [4]:
outcome = "demand"
treatment = "treatment"
col = list(train_data.columns)
col.remove("price")
print(col)

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove('income')
print(cov)

features = col[:]
features.remove(outcome)

print(features)

['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'demand', 'treatment']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'treatment']


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train_data, test_size=0.1)

In [6]:
## Fit treatment
model_t = LogisticRegression()
model_t.fit(train_data[cov], train_data[treatment])

train_data['p1'] = model_t.predict_proba(train_data[cov])[:,1]
train_data['p0'] = 1 - train_data['p1']

train_data["prediction"] = np.where(train_data["p1"] >= 0.5, 1, 0)

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
model_y.fit(train_data[features], train_data[outcome])

    

GradientBoostingRegressor(n_estimators=1000, random_state=0)

In [7]:
train_data.shape

(9000, 15)

## Optimization

In [8]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

def fitness_function(propensity_score, solution_idx):
    policy = np.where(propensity_score >= 0.5, 1, 0)
    return np.mean(revenue_fn(train_data,  0., 0.15, 1, policy))


In [9]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,treatment,p1,p0,prediction
5466,2,28,8.508083,6,9,1,0,6.093560,1.878202,0.85,26.554042,0,0.446247,0.553753,0
9435,1,54,9.578934,4,14,0,1,5.795057,1.480526,1.00,21.789467,1,0.417497,0.582503,0
8443,3,33,4.086286,7,8,0,1,4.088569,1.020977,1.00,24.043143,1,0.464931,0.535069,0
1483,4,54,3.762321,6,9,1,0,7.755960,0.370185,0.85,13.281160,0,0.428216,0.571784,0
9227,3,20,3.852820,4,6,1,1,3.956176,1.761313,1.00,18.926410,1,0.453593,0.546407,0


In [10]:
#0: discount, level 2, 1: no discount - level 1
np.mean(revenue_fn(train_data,  0.0, 0.15, 1, train_data.treatment.values))

14.308830571002158

In [11]:
np.mean(train_data['demand']*train_data['price'])

14.377760848779936

## PyGrad

In [12]:
import pygad
import numpy

function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 2000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 20 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Running model
Generation = 1
Fitness    = 13.710571892440957
Change     = 13.710571892440957


Generation = 2
Fitness    = 13.721454545184855
Change     = 0.010882652743898547


Generation = 3
Fitness    = 13.73447167612647
Change     = 0.013017130941614496


Generation = 4
Fitness    = 13.752954217738072
Change     = 0.018482541611602343


Generation = 5
Fitness    = 13.762080202130889
Change     = 0.009125984392817088


Generation = 6
Fitness    = 13.766784419325983
Change     = 0.004704217195094174


Generation = 7
Fitness    = 13.78465083116337
Change     = 0.01786641183738702


Generation = 8
Fitness    = 13.785493413737132
Change     = 0.0008425825737621295


Generation = 9
Fitness    = 13.794879975667365
Change     = 0.009386561930233128


Generation = 10
Fitness    = 13.802424666498185
Change     = 0.007544690830819434


Generation = 11
Fitness    = 13.803686714385142
Change     = 0.0012620478869571627


Generation = 12
Fitness    = 13.814233836152216
Change     = 0.01054712176

Change     = 0.006293421137268496


Generation = 104
Fitness    = 14.078934704311818
Change     = 0.001958006454815475


Generation = 105
Fitness    = 14.078934704311818
Change     = 0.0


Generation = 106
Fitness    = 14.079248539087834
Change     = 0.00031383477601565346


Generation = 107
Fitness    = 14.08142320058482
Change     = 0.0021746614969870848


Generation = 108
Fitness    = 14.085441186105886
Change     = 0.004017985521064915


Generation = 109
Fitness    = 14.086886163148533
Change     = 0.001444977042647011


Generation = 110
Fitness    = 14.086886163148533
Change     = 0.0


Generation = 111
Fitness    = 14.088615184514905
Change     = 0.0017290213663727627


Generation = 112
Fitness    = 14.088615184514905
Change     = 0.0


Generation = 113
Fitness    = 14.088615184514905
Change     = 0.0


Generation = 114
Fitness    = 14.091595411615678
Change     = 0.0029802271007728365


Generation = 115
Fitness    = 14.091595411615678
Change     = 0.0


Generation = 116
Fitness 

Change     = 0.0014933066935594042


Generation = 210
Fitness    = 14.178904385304797
Change     = 0.0011308658482551692


Generation = 211
Fitness    = 14.178904385304797
Change     = 0.0


Generation = 212
Fitness    = 14.178904385304797
Change     = 0.0


Generation = 213
Fitness    = 14.178904385304797
Change     = 0.0


Generation = 214
Fitness    = 14.18485134626998
Change     = 0.005946960965182768


Generation = 215
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 216
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 217
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 218
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 219
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 220
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 221
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 222
Fitness    = 14.18485134626998
Change     = 0.0


Generation = 223
Fitness    = 14.18485134626998
Chang

Change     = 0.0


Generation = 322
Fitness    = 14.226986795481354
Change     = 0.0008250992499334586


Generation = 323
Fitness    = 14.226986795481354
Change     = 0.0


Generation = 324
Fitness    = 14.226986795481354
Change     = 0.0


Generation = 325
Fitness    = 14.226986795481354
Change     = 0.0


Generation = 326
Fitness    = 14.226986795481354
Change     = 0.0


Generation = 327
Fitness    = 14.226986795481354
Change     = 0.0


Generation = 328
Fitness    = 14.227992138661875
Change     = 0.0010053431805214785


Generation = 329
Fitness    = 14.227992138661875
Change     = 0.0


Generation = 330
Fitness    = 14.227992138661875
Change     = 0.0


Generation = 331
Fitness    = 14.227992138661875
Change     = 0.0


Generation = 332
Fitness    = 14.230084321241305
Change     = 0.002092182579430002


Generation = 333
Fitness    = 14.230084321241305
Change     = 0.0


Generation = 334
Fitness    = 14.230084321241305
Change     = 0.0


Generation = 335
Fitness    = 14.23008432124

Fitness    = 14.257065168687244
Change     = 1.6930083916122385e-05


Generation = 436
Fitness    = 14.257065168687244
Change     = 0.0


Generation = 437
Fitness    = 14.257322405224615
Change     = 0.000257236537370531


Generation = 438
Fitness    = 14.257322405224615
Change     = 0.0


Generation = 439
Fitness    = 14.257322405224615
Change     = 0.0


Generation = 440
Fitness    = 14.257322405224615
Change     = 0.0


Generation = 441
Fitness    = 14.257322405224615
Change     = 0.0


Generation = 442
Fitness    = 14.257797153589827
Change     = 0.0004747483652121076


Generation = 443
Fitness    = 14.258199744480997
Change     = 0.0004025908911700071


Generation = 444
Fitness    = 14.258199744480997
Change     = 0.0


Generation = 445
Fitness    = 14.258918534755319
Change     = 0.0007187902743215346


Generation = 446
Fitness    = 14.258918534755319
Change     = 0.0


Generation = 447
Fitness    = 14.259359333322974
Change     = 0.00044079856765577574


Generation = 448
Fitness

...

Change     = 0.0


Generation = 1482
Fitness    = 14.37481483026748
Change     = 0.0


Generation = 1483
Fitness    = 14.37481483026748
Change     = 0.0


Generation = 1484
Fitness    = 14.37481483026748
Change     = 0.0


Generation = 1485
Fitness    = 14.37481483026748
Change     = 0.0


Generation = 1486
Fitness    = 14.37481483026748
Change     = 0.0


Generation = 1487
Fitness    = 14.37614373305879
Change     = 0.001328902791309261


Generation = 1488
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1489
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1490
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1491
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1492
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1493
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1494
Fitness    = 14.37614373305879
Change     = 0.0


Generation = 1495
Fitness    = 14.37614373305879
Change     = 0.0


Generation =

Change     = 0.0


Generation = 1599
Fitness    = 14.38247683207027
Change     = 0.0


Generation = 1600
Fitness    = 14.38247683207027
Change     = 0.0


Generation = 1601
Fitness    = 14.38247683207027
Change     = 0.0


Generation = 1602
Fitness    = 14.38272734964268
Change     = 0.00025051757241101313


Generation = 1603
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1604
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1605
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1606
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1607
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1608
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1609
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1610
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1611
Fitness    = 14.38272734964268
Change     = 0.0


Generation = 1612
Fitness    = 14.38272734964268
Change     = 0.0


Generation

Change     = 0.0


Generation = 1717
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1718
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1719
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1720
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1721
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1722
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1723
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1724
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1725
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1726
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1727
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1728
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1729
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 1730
Fitness    = 14.389120703417866
Change     = 0.0


Generation = 17

Change     = 0.0005826830536488359


Generation = 1835
Fitness    = 14.391801939241608
Change     = 0.0


Generation = 1836
Fitness    = 14.391801939241608
Change     = 0.0


Generation = 1837
Fitness    = 14.391801939241608
Change     = 0.0


Generation = 1838
Fitness    = 14.391801939241608
Change     = 0.0


Generation = 1839
Fitness    = 14.391801939241608
Change     = 0.0


Generation = 1840
Fitness    = 14.391801939241608
Change     = 0.0


Generation = 1841
Fitness    = 14.392113505065282
Change     = 0.00031156582367408703


Generation = 1842
Fitness    = 14.392113505065282
Change     = 0.0


Generation = 1843
Fitness    = 14.393763949826257
Change     = 0.0016504447609744233


Generation = 1844
Fitness    = 14.393763949826257
Change     = 0.0


Generation = 1845
Fitness    = 14.393763949826257
Change     = 0.0


Generation = 1846
Fitness    = 14.393763949826257
Change     = 0.0


Generation = 1847
Fitness    = 14.393763949826257
Change     = 0.0


Generation = 1848
Fitness    

Change     = 0.0


Generation = 1952
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1953
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1954
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1955
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1956
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1957
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1958
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1959
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1960
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1961
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1962
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1963
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1964
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 1965
Fitness    = 14.398009935659255
Change     = 0.0


Generation = 19

In [24]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [ 3.31533982  4.6518709  -3.48106058 ... -5.4188069  -7.77434956
 -3.36895737]
Fitness value of the best solution = 14.398257623545387


In [25]:
proposed_policy = np.where(solution >= 0.5, 1, 0)

In [26]:
proposed_policy

array([1, 1, 0, ..., 0, 0, 0])

In [27]:
discount_all = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.zeros(train_data.shape[0])))
no_discount = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.ones(train_data.shape[0])))

In [28]:
print("Current policy {:.4f}".format(discount_all))
print("Discount all {:.4f}".format(discount_all))
print("No discount {:.4f}".format(no_discount))
print("-"*100)
print("Proposed strategy {:.4f}".format(solution_fitness))

Current policy 13.4525
Discount all 13.4525
No discount 14.3125
----------------------------------------------------------------------------------------------------
Proposed strategy 14.3983


In [29]:
from collections import Counter 
print(Counter(proposed_policy))
print(Counter(train_data.treatment.values))


Counter({1: 4922, 0: 4078})
Counter({0: 5125, 1: 3875})


In [30]:
current_policy = train_data.treatment.values

In [31]:
proposed_policy = [str(x) for x in proposed_policy]
current_policy = [str(x) for x in current_policy]

In [59]:
best = ga_instance.best_solution()

In [62]:
a = np.array([best[0]]*1000)
a.shape


(1000, 9000)

In [63]:
init = np.array([best[0]]*1000)

In [ ]:
function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 90 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0


def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(initial_population = init,
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()

Running model
Generation = 1
Fitness    = 14.398257623545387
Change     = 14.398257623545387


Generation = 2
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 3
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 4
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 5
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 6
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 7
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 8
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 9
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 10
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 11
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 12
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 13
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 14
Fitness    = 14.398257623545387
Change     = 0.0


Generation = 15
Fitness    = 14.3982576235